# Get all files names in the direcotry

In [5]:
import os
import csv

def list_subdirectories(directory_path):
    return [os.path.join(directory_path, d, "nu_T1_brain_6dof2Ref.nii.gz") for d in os.listdir(directory_path) ]

def write_to_csv(subdirectories):
    with open('biobank.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['img_path'])  # Column header
        for subdir in subdirectories:
            csvwriter.writerow([subdir])

# Example usage:
directory_path = "/scratch1/ajoshi/BrainAge_Preproc_FS7.1.1/"
subdirs = list_subdirectories(directory_path)
write_to_csv(subdirs)


In [3]:
import os

def k_fold_cross_validation_from_csv(input_file, column_name, k, output_dir):
    """
    Splits the data from the specified column of the input CSV file into k folds for cross-validation 
    and writes them to separate CSV files.
    
    Parameters:
    - input_file: Path to the input CSV file.
    - column_name: The name of the column from which to select samples.
    - k: Number of folds for cross-validation.
    - output_dir: Directory where the output CSV files will be saved.
    """
    
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Check if the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in {input_file}")
    
    # Shuffle the data
    df = df.sample(frac=1).reset_index(drop=True)
    
    # Split the data into k parts
    fold_size = len(df) // k
    folds = [df.iloc[i * fold_size: (i + 1) * fold_size] for i in range(k)]
    
    # Handle any remaining samples if the dataset size is not perfectly divisible by k
    for i in range(len(df) % k):
        folds[i] = folds[i].append(df.iloc[-(i+1)])
    
    # For each fold, save the training and test sets to separate files
    for i in range(k):
        test = folds[i]
        train = pd.concat([folds[j] for j in range(k) if j != i], ignore_index=True)
        
        test_file = os.path.join(output_dir, f"Biobank_test_fold_{i+1}.csv")
        train_file = os.path.join(output_dir, f"Biobank_train_fold_{i+1}.csv")
        
        test[[column_name]].to_csv(test_file, index=False)
        train[[column_name]].to_csv(train_file, index=False)

# For demonstration, we will not run the function yet.
# k_fold_cross_validation_from_csv('biobank.csv', 'img_path', 5, 'output_directory_path')


In [4]:
# Read the CSV file
import pandas as pd
#csv_path = 'biobank.csv'
#df = pd.read_csv(csv_path)
k_fold_cross_validation_from_csv('biobank.csv', 'img_path', 5, './')




/tmp/SLURM_16829085/ipykernel_8682/512982690.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[i] = folds[i].append(df.iloc[-(i+1)])
/tmp/SLURM_16829085/ipykernel_8682/512982690.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[i] = folds[i].append(df.iloc[-(i+1)])
/tmp/SLURM_16829085/ipykernel_8682/512982690.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[i] = folds[i].append(df.iloc[-(i+1)])
/tmp/SLURM_16829085/ipykernel_8682/512982690.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[i] = folds[i].append(df.iloc[-(i+1)])


# Generate a subset

In [7]:
import pandas as pd
import random

def random_sample_from_csv(input_file, column_name, n, output_file, exclude_file):
    """
    Randomly selects n samples from the specified column of the input CSV file
    and writes them to a new CSV file.
    
    Parameters:
    - input_file: Path to the input CSV file.
    - column_name: The name of the column from which to select samples.
    - n: The number of random samples to select.
    - output_file: Path to the output CSV file.
    - exclude_file: Path to the CSV file containing image paths to exclude.
    """
    
    # Read the CSV files
    df = pd.read_csv(input_file)
    print(len(df))
    exclude_df = pd.read_csv(exclude_file)
    
    # Convert the exclude DataFrame to a list
    exclude_list = exclude_df[column_name].tolist()
    
    # Filter out the rows with image paths that are in the exclude list
    filtered_df = df[~df[column_name].isin(exclude_list)]
    print(len(filtered_df))
    # Check if the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in {input_file}")
    
    # Check if there are enough rows to sample
    if n > len(filtered_df):
        raise ValueError(f"Cannot sample {n} rows from a dataset of {len(filtered_df)} rows after excluding")
    
    # Randomly sample n rows from the column of the filtered DataFrame
    samples = random.sample(list(filtered_df[column_name]), n)
    
    # Convert samples to DataFrame
    samples_df = pd.DataFrame(samples, columns=[column_name])
    
    # Write to the output CSV file
    samples_df.to_csv(output_file, index=False)

random_sample_from_csv('biobank.csv', 'img_path', 100, 'subset_val_biobank.csv', 'subset_biobank.csv')


45464
40464


# Copy anr rename subset

In [6]:


import pandas as pd
import shutil
import os

# Read the CSV file
csv_path = 'subset_val_biobank.csv'
df = pd.read_csv(csv_path)

# Specify the column name containing the file paths
column_name = 'img_path'

# Create the T1 directory if it doesn't exist
dir = '/scratch1/akrami/Latest_Data/Data_pre/BioBank_val/t1/'

# Process each file path
for file_path in df[column_name]:
    # Check if the file exists
   
    if os.path.exists(file_path):
        # Extract the subname from the file path
        full_subname = os.path.basename(os.path.dirname(file_path))
        # Extract only the desired part of the subname (before the first underscore)
        main_subname = full_subname.split('_')[0]
        # Define the destination path
        dest_path_sub = f'{main_subname}_t1.nii.gz'
        dest_path = dir+ dest_path_sub
        # Copy the file to the destination
        shutil.copy(file_path, dest_path)


# files which are not there

In [ ]:
import pandas as pd
import shutil
import os

# Read the CSV file
csv_path = 'subset_biobank.csv'
df = pd.read_csv(csv_path)

# Specify the column name containing the file paths
column_name = 'img_path'

# Create the T1 directory if it doesn't exist
dir = '/scratch1/akrami/Latest_Data/Data_pre/BioBank/t1/'

# Process each file path
for file_path in df[column_name]:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(file_path)
    # if os.path.exists(file_path):
    #     # Extract the subname from the file path
    #     full_subname = os.path.basename(os.path.dirname(file_path))
    #     # Extract only the desired part of the subname (before the first underscore)
    #     main_subname = full_subname.split('_')[0]
    #     # Define the destination path
    #     dest_path_sub = f'{main_subname}_t1.nii.gz'
    #     dest_path = dir+ dest_path_sub
    #     # Copy the file to the destination
    #     shutil.copy(file_path, dest_path)


In [2]:
import pandas as pd

# Read the source CSV
source_csv_path = "/project/ajoshi_27/akrami//patched-Diffusion-Models-UAD/Data/splits/BioBank_train.csv"
df = pd.read_csv(source_csv_path, nrows=200)

# Write the first 200 rows to a new CSV
output_csv_path = "/project/ajoshi_27/akrami//patched-Diffusion-Models-UAD/Data/splits/BioBank_small.csv"
df.to_csv(output_csv_path, index=False)
